# BTC: ProbStates Strategy Backtest

Этот ноутбук демонстрирует профессиональный бэктест стратегии на Bitcoin с использованием ProbStates:
- мэппинг сотен/тысяч гетерогенных сигналов в вероятности (L2) и фазы (L4)
- робастная агрегация ⊕₄ (режим 'weight')
- стресс‑тесты под шумами
- метрики: CAGR, Sharpe, максимальная просадка

Внимание: это исследовательский материал, не является инвестиционной рекомендацией.


In [ ]:
# !pip install yfinance pandas numpy matplotlib --quiet
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

from probstates.markets import (
    sma, momentum, rsi, indicator_to_prob, sentiment_to_phase,
    FeatureSpec, aggregate_specs, btc_signal_from_arrays,
)
from probstates import set_phase_or_mode
from probstates.coherence import dephase, amp_damp

plt.rcParams['figure.figsize'] = (12, 5)



In [ ]:
# Load BTC-USD daily data
start, end = '2018-01-01', '2025-01-01'
df = yf.download('BTC-USD', start=start, end=end, progress=False)
df = df[['Close']].dropna().copy()

df['mom10'] = momentum(df['Close'].values, 10)
df['rsi14'] = rsi(df['Close'].values, 14)
df['sma20'] = sma(df['Close'].values, 20)
df['sma50'] = sma(df['Close'].values, 50)

df.dropna(inplace=True)
df.head()


In [ ]:
def build_specs(row) -> list:
    specs = []
    # momentum
    p_mom = float(indicator_to_prob(np.array([row['mom10']]))[-1])
    phi_mom = 0.0 if row['mom10'] >= 0 else np.pi
    specs.append(FeatureSpec('momentum_10', p_mom, phi_mom, weight=2))
    # rsi
    p_rsi = float(indicator_to_prob(np.array([row['rsi14']]), center=50.0, k=1.2)[-1])
    phi_rsi = 0.0 if row['rsi14'] >= 50 else np.pi
    specs.append(FeatureSpec('rsi_14', p_rsi, phi_rsi, weight=1))
    # ma cross
    cross = float(row['sma20'] - row['sma50'])
    p_cross = float(indicator_to_prob(np.array([cross]))[-1])
    phi_cross = 0.0 if cross >= 0 else np.pi
    specs.append(FeatureSpec('sma20_vs_50', p_cross, phi_cross, weight=2))
    return specs

set_phase_or_mode('weight')

agg_p = []
for _, r in df.iterrows():
    agg = aggregate_specs(build_specs(r), mode='weight')
    agg_p.append(agg.probability)

df['agg_p'] = agg_p
df[['Close','agg_p']].tail()


In [ ]:
# Simple rules: buy if p>=0.55, sell if p<=0.45, else hold
buy_thr, sell_thr = 0.55, 0.45
pos = 0.0
positions = []
for p in df['agg_p'].values:
    if p >= buy_thr:
        pos = 1.0
    elif p <= sell_thr:
        pos = 0.0
    positions.append(pos)

df['pos'] = positions

# Backtest with fees/slippage
fee = 0.001  # 10 bps per trade
ret = df['Close'].pct_change().fillna(0.0)
trade = np.abs(np.diff([0.0, *df['pos'].values]))
strategy_ret = df['pos'].shift(1).fillna(0.0) * ret - fee * trade

def metrics(series: pd.Series) -> dict:
    x = series.dropna().values
    ann = np.prod(1.0 + x) ** (252/len(x)) - 1.0
    vol = np.std(x) * np.sqrt(252)
    sharpe = ann / (vol + 1e-12)
    curve = (1.0 + x).cumprod()
    peak = np.maximum.accumulate(curve)
    mdd = float(np.max((peak - curve)/peak))
    return dict(CAGR=ann, Sharpe=sharpe, MDD=mdd)

bench = ret  # buy&hold
m_strat = metrics(strategy_ret)
m_bench = metrics(bench)
print('Strategy:', {k: round(v,3) for k,v in m_strat.items()})
print('Buy&Hold:', {k: round(v,3) for k,v in m_bench.items()})

(1.0 + bench).cumprod().plot(label='Buy&Hold')
(1.0 + strategy_ret).cumprod().plot(label='ProbStates')
plt.legend(); plt.title('Equity curves'); plt.show()


In [ ]:
# Robustness stress: phase/noise sweeps
sigmas = [0.0, 0.05, 0.1, 0.2]
fees = [0.0005, 0.001, 0.002]
res = []
for s in sigmas:
    for f in fees:
        pos = 0.0
        eq = 1.0
        last_p = 0.0
        for p in df['agg_p'].values:
            # noisy probability via simple damping proxy
            p_noisy = float(amp_damp(dephase(PhaseState(p,0.0), s), alpha=s/2).probability)
            if p_noisy >= buy_thr and last_p < 1.0:
                eq *= (1.0 - f)
                last_p = 1.0
            elif p_noisy <= sell_thr and last_p > 0.0:
                eq *= (1.0 - f)
                last_p = 0.0
        res.append((s, f, eq))

res[:5]
